## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
0,0,Bluff,-46.6000,168.3333,53.80,76,100,10.92,NZ,overcast clouds,2022-01-25 23:44:06
1,1,Gat,31.6100,34.7642,50.40,100,9,10.38,IL,clear sky,2022-01-25 23:44:07
2,2,Yuli,9.6989,10.2744,69.87,18,79,10.38,NG,broken clouds,2022-01-25 23:44:07
3,3,Vaini,-21.2000,-175.2000,84.36,79,20,10.36,TO,few clouds,2022-01-25 23:41:06
4,4,Ruwi,23.5996,58.5507,62.29,65,0,5.86,OM,clear sky,2022-01-25 23:44:07


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 66
What is the maximum temperature you would like for your trip? 88


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count().isnull()

City_ID                False
City                   False
Lat                    False
Lng                    False
Max Temp               False
Humidity               False
Cloudiness             False
Wind Speed             False
Country                False
Current Description    False
Date                   False
dtype: bool

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.count().dropna()
clean_df = preferred_cities_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Yuli,NG,69.87,broken clouds,9.6989,10.2744,
3,Vaini,TO,84.36,few clouds,-21.2000,-175.2000,
5,Neuquen,AR,87.69,few clouds,-38.9516,-68.0591,
7,Arraial Do Cabo,BR,76.96,clear sky,-22.9661,-42.0278,
8,Carnarvon,AU,77.07,light rain,-24.8667,113.6333,
10,Axim,GH,77.67,clear sky,4.8699,-2.2405,
11,Mitsamiouli,KM,81.27,overcast clouds,-11.3847,43.2844,
12,Hilo,US,77.11,few clouds,19.7297,-155.0900,
19,Ippy,CF,78.62,overcast clouds,6.2679,21.2247,
22,Ugoofaaru,MV,79.79,overcast clouds,5.6667,73.0000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key  
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    #hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Yuli,NG,69.87,broken clouds,9.6989,10.2744,
3,Vaini,TO,84.36,few clouds,-21.2000,-175.2000,Keleti Beach Resort
5,Neuquen,AR,87.69,few clouds,-38.9516,-68.0591,Hotel del Comahue
7,Arraial Do Cabo,BR,76.96,clear sky,-22.9661,-42.0278,Pousada Porto Praia
8,Carnarvon,AU,77.07,light rain,-24.8667,113.6333,Hospitality Carnarvon


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.tail()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
716,Port Keats,AU,79.00,light rain,-14.2500,129.5500,TDC Village (Hundred Man Camp)
718,Bahia De Caraquez,EC,75.38,light rain,-0.6000,-80.4167,Saiananda Eco-Hotel
720,Yumbing,PH,79.66,overcast clouds,9.2393,124.6578,Medano Island Resort
721,Ahuimanu,US,80.31,broken clouds,21.4447,-157.8378,Paradise Bay Resort
722,Puerto Escondido,MX,83.23,clear sky,15.8500,-97.0667,Aldea del Bazar Hotel and Spa


In [11]:
clean_hotel_df["Hotel Name"].unique()

array(['', 'Keleti Beach Resort', 'Hotel del Comahue',
       'Pousada Porto Praia', 'Hospitality Carnarvon',
       'Axim Beach Resort And Spa', 'Foyer ADM', 'Hilo Hawaiian Hotel',
       'Ungoofaaru Inn', 'M.O.S. Logística - Filial Fortaleza',
       'Southern Sun Waterfront Cape Town', 'Nuevo Hotel Plaza',
       'Casa Tia Villa', 'Hotel On The Rock by Prasanthi',
       'The Halyards Hotel', 'Tu Casa', 'Scoop Guest House',
       'ZINGARA 76ft Catamaran, full crew-all included', 'Paradise Inn',
       'Hotel Boutique Quinta das Videiras', 'Hotel Castelo',
       'Pereybere Hotel & Spa', 'Pousada Maria Bonita',
       'Divi Village Golf & Beach Resort', 'Frutillares Centro Coihueco',
       'People ThankYou', 'Enviro Villa',
       'Holiday Inn Express Lompoc, an IHG Hotel', 'SEGAYA PROPERTY',
       'Redface Studios Meublés', 'Casa de Isabella', 'Rio Xingu Camping',
       'Hotel Mirador', '39 On Nile Guest House',
       'Finch Bay Galapagos Hotel', 'Wasai Hotel',
       'Hotel Te

In [12]:
clean_hotel_df.shape

(311, 7)

In [13]:
hotel_df.shape

(314, 7)

In [14]:
clean_hotel_df = clean_hotel_df[clean_hotel_df["Hotel Name"] != ""]

In [15]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Vaini,TO,84.36,few clouds,-21.2000,-175.2000,Keleti Beach Resort
5,Neuquen,AR,87.69,few clouds,-38.9516,-68.0591,Hotel del Comahue
7,Arraial Do Cabo,BR,76.96,clear sky,-22.9661,-42.0278,Pousada Porto Praia
8,Carnarvon,AU,77.07,light rain,-24.8667,113.6333,Hospitality Carnarvon
10,Axim,GH,77.67,clear sky,4.8699,-2.2405,Axim Beach Resort And Spa


In [16]:
clean_hotel_df.tail()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
716,Port Keats,AU,79.00,light rain,-14.2500,129.5500,TDC Village (Hundred Man Camp)
718,Bahia De Caraquez,EC,75.38,light rain,-0.6000,-80.4167,Saiananda Eco-Hotel
720,Yumbing,PH,79.66,overcast clouds,9.2393,124.6578,Medano Island Resort
721,Ahuimanu,US,80.31,broken clouds,21.4447,-157.8378,Paradise Bay Resort
722,Puerto Escondido,MX,83.23,clear sky,15.8500,-97.0667,Aldea del Bazar Hotel and Spa


In [17]:
# 8a. Create the output File (CSV)
output_data_file =" WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map.

max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))